[![Open In Colab](https://colab.research.googleusercontent.com/assets/colab-badge.svg)](https://colab.research.google.com)

Si tu repo está en GitHub, usa este enlace editando USER/REPO/BRANCH:
[Open in Colab (GitHub)](https://colab.research.google.com/github/USER/REPO/blob/BRANCH/professor/pandas_v2/04_data_cleaning_flow_v2.ipynb)



# 04 v2 Limpieza de datos end-to-end (versión detallada)

Objetivos:
- Construir datos "sucios" realistas y planificar su limpieza.
- Normalizar tipos (texto, numéricos, fechas), tratar nulos y duplicados.
- Homologar categorías con catálogos (joins) y validar reglas.
- Guardar un dataset limpio para análisis/EDA.

Nota: Cada celda de código es corta y va precedida de una explicación.


## Preparación e importaciones

Usaremos `pandas` (`pd`), `numpy` (`np`) y `pathlib.Path` para rutas. En la celda de abajo:
- `Path(...).mkdir(parents=True, exist_ok=True)` crea la carpeta de salida si no existe.
- Mostramos versiones con `pd.__version__`.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

print("Versiones:")
print("pandas=", pd.__version__)

OUT_DIR = Path("professor/pandas_v2/data/clean")
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("Salida:", OUT_DIR.as_posix())


## Creamos datos "sucios" (ventas y clientes)

Usamos `pd.DataFrame({...})` para construir ejemplos con errores reales:
- Fechas con formatos mixtos, montos como texto con símbolos y espacios.
- Espacios y mayúsculas/minúsculas inconsistentes en texto.
- Duplicados en `venta_id` y `cliente_id`.


In [ ]:
ventas_raw = pd.DataFrame({
    "venta_id": [1, 2, 2, 3, 4, 5],
    "cliente_id": [101, 102, 102, 103, None, 104],
    "fecha": [
        "2024-01-01", "2024/01/02", "02-01-2024", "2024-13-01", None, "2024-01-05 10:00"
    ],
    "monto": ["$100", "200.5 ", "dos", None, "-5", " 300"],
    "categoria": [" A ", "b", "B", None, "A", "b"],
    "canal": ["Web", " tienda ", "Tienda", "WEB", "?", "web"],
})

clientes_raw = pd.DataFrame({
    "cliente_id": [101, 102, 102, 105],
    "nombre": [" Ana ", "luis", "LUIS", "Mara"],
    "pais": [" mx ", "MX", "mx", "US"],
})

print("Ventas sucias:")
print(ventas_raw)
print("\nClientes sucios:")
print(clientes_raw)


## Vistazo rápido (head, info)

Usamos `DataFrame.head()` para una muestra rápida y `DataFrame.info()` para ver tipos y nulos por columna.


In [ ]:
print("ventas_raw.head():")
print(ventas_raw.head())
print("\nclientes_raw.head():")
print(clientes_raw.head())
print("\nInfo ventas_raw:")
print(ventas_raw.info())
print("\nInfo clientes_raw:")
print(clientes_raw.info())


## Normalización de texto

Funciones usadas en la celda siguiente:
- `Series.astype(str)` para garantizar tipo string.
- `Series.str.strip()` para quitar espacios al inicio/fin.
- `Series.str.title()`, `Series.str.upper()`, `Series.str.lower()` para homogenizar casing.


In [ ]:
ventas = ventas_raw.copy()
clientes = clientes_raw.copy()

clientes["nombre"] = clientes["nombre"].astype(str).str.strip().str.title()
clientes["pais"] = clientes["pais"].astype(str).str.strip().str.upper()

ventas["categoria"] = ventas["categoria"].astype(str).str.strip().str.lower()
ventas["canal"] = ventas["canal"].astype(str).str.strip().str.lower()

print("Texto normalizado (clientes):\n", clientes.head(), sep="")
print("\nTexto normalizado (ventas):\n", ventas.head(), sep="")


## Conversión de tipos (numérico y fecha)

Funciones usadas abajo:
- `Series.str.replace()` para remover símbolos y espacios.
- `pd.to_numeric(..., errors='coerce')` para convertir `monto` a número.
- `pd.to_datetime(..., errors='coerce', utc=True)` para parsear y normalizar `fecha` a UTC.


In [ ]:
ventas["monto"] = ventas["monto"].astype(str).str.replace("$", "", regex=False).str.strip()
ventas["monto"] = pd.to_numeric(ventas["monto"], errors="coerce")
ventas["fecha"] = pd.to_datetime(ventas["fecha"], errors="coerce", utc=True)

print("Montos y fechas convertidos:")
print(ventas[["monto", "fecha"]])


## Nulos: diagnóstico y tratamiento

Funciones usadas abajo:
- `DataFrame.isna().sum()` para conteo de nulos.
- Asignación condicional con `DataFrame.loc[cond, col] = ...`.
- `Series.replace({...})` para marcar valores especiales como `NaN`.
- `Series.fillna(...)` para imputar valores faltantes.


In [ ]:
print("Conteo de nulos antes:")
print(ventas.isna().sum())

ventas.loc[ventas["monto"] < 0, "monto"] = np.nan
ventas["canal"] = ventas["canal"].replace({"?": np.nan})

print("\nImputamos 'canal' faltante con 'web' (ejemplo simple):")
ventas["canal"] = ventas["canal"].fillna("web")

print("\nConteo de nulos después:")
print(ventas.isna().sum())


## Reemplazos y estandarización

Usamos `Series.replace({...})` para mapear valores a categorías estándar y para tratar variantes (`"a"→"A"`, `"b"→"B"`). Si hubiera patrones, puede usarse `regex=True`.


In [ ]:
ventas["categoria"] = ventas["categoria"].replace({np.nan: "desconocido"})
ventas["categoria"] = ventas["categoria"].replace({"a": "A", "b": "B", "none": "desconocido"})

print(ventas[["categoria", "canal"]].head())


## Catálogo y homologación por join

En la celda de abajo:
- Construimos `cat_map` con `pd.DataFrame`.
- Homologamos con `pd.merge(..., how='left', indicator=True)` para ver `_merge` (`left_only/right_only/both`).


In [ ]:
cat_map = pd.DataFrame({
    "categoria": ["A", "B", "desconocido"],
    "categoria_final": ["A", "B", "DESCONOCIDO"],
})

merged_cats = pd.merge(
    ventas[["venta_id", "categoria"]],
    cat_map,
    on="categoria",
    how="left",
    indicator=True,
)
print("Resultado de join de categorías (primeras filas):")
print(merged_cats.head())
print("\nConteo por _merge:")
print(merged_cats["_merge"].value_counts())

ventas = ventas.merge(cat_map, on="categoria", how="left")


## Enriquecimiento con clientes (left join) y auditoría

Usamos `pd.merge(ventas, clientes, on='cliente_id', how='left', indicator=True)` para enriquecer y auditar con `_merge` cuántas filas no hicieron match (`left_only`).


In [ ]:
ventas_enr = pd.merge(
    ventas,
    clientes,
    on="cliente_id",
    how="left",
    indicator=True,
)
print("Resultado de enriquecimiento (primeras filas):")
print(ventas_enr.head())
print("\nConteo por _merge (espera left_only si cliente_id faltó o no existía en clientes):")
print(ventas_enr["_merge"].value_counts())


## Duplicados y llaves

Funciones usadas abajo:
- `Series.duplicated()` para detectar duplicados.
- `DataFrame.drop_duplicates(subset=[...], keep='first')` para deduplicar por llave.


In [ ]:
print("Duplicados en ventas por venta_id:")
print(ventas_enr["venta_id"].duplicated().value_counts())

ventas_enr = ventas_enr.drop_duplicates(subset=["venta_id"], keep="first")
print("\nVentas tras drop_duplicates:")
print(len(ventas_enr))


## Validaciones y reglas de negocio

Funciones usadas abajo:
- `Series.clip(lower=0)` para cortar valores negativos.
- `DataFrame.dropna(subset=[...])` para exigir presencia de campos críticos (p. ej., `fecha`).


In [ ]:
ventas_enr["monto"] = ventas_enr["monto"].clip(lower=0)
registros_con_fecha = ventas_enr["fecha"].notna().sum()
print("Registros con fecha válida:", registros_con_fecha)

# Si consideras obligatorio 'fecha' para toda venta, filtra:
ventas_validas = ventas_enr.dropna(subset=["fecha"])  # decisión explícita
print("Ventas válidas (sin NaT en fecha):", len(ventas_validas))


## Outliers (percentiles y `clip`)

Herramientas sugeridas para la celda siguiente (si decides aplicarlo):
- `Series.quantile([0.01, 0.99])` para estimar umbrales p1–p99.
- `Series.clip(lower=..., upper=...)` para recortar extremos de `monto`.


## Deduplicación de clientes y `validate`

Funciones usadas abajo:
- `DataFrame.drop_duplicates(subset=[...])` para deduplicar catálogo de clientes.
- `pd.merge(..., validate='m:1')` para asegurar relación muchas-ventas-a-un-cliente sin duplicaciones inesperadas.


In [ ]:
clientes_dedup = clientes.drop_duplicates(subset=["cliente_id"], keep="first")
print("Clientes antes/after dedup:")
print(len(clientes), len(clientes_dedup))

ventas_enr_v = pd.merge(
    ventas,
    clientes_dedup,
    on="cliente_id",
    how="left",
    validate="m:1",
    indicator=True,
)
print("\nEnriquecimiento validado m:1 (primeras filas):")
print(ventas_enr_v.head())
print("\n_merge counts:")
print(ventas_enr_v["_merge"].value_counts())


## Resumen de calidad y validaciones rápidas

Funciones usadas abajo:
- `Series.is_unique` para validar unicidad de `venta_id`.
- `DataFrame.isna().sum()` para conteo de nulos en campos clave.
- `Series.value_counts(dropna=False)` para distribuciones.


In [ ]:
print("Unicidad de venta_id (espera True):", ventas_enr_v["venta_id"].is_unique)
print("\nNulos en campos clave:")
print(ventas_enr_v[["venta_id", "cliente_id", "fecha", "monto"]].isna().sum())

print("\nDistribución de canal:")
print(ventas_enr_v["canal"].value_counts(dropna=False))

print("\nDistribución de categoria_final:")
print(ventas_enr_v["categoria_final"].value_counts(dropna=False))


## Guardado de datos limpios

Funciones usadas abajo:
- `DataFrame.sort_values(...)` para ordenar al exportar.
- `DataFrame.to_csv(...)` y `DataFrame.to_parquet(...)` para persistencia eficiente.
- `Path` para construir rutas de salida.


In [ ]:
cols_finales = [
    "venta_id", "cliente_id", "fecha", "monto",
    "categoria_final", "canal", "nombre", "pais"
]
limpio_v2 = ventas_enr_v[cols_finales].sort_values(["cliente_id", "fecha", "venta_id"], na_position="last")

csv_path = OUT_DIR / "dataset_limpio_v2.csv"
parquet_path = OUT_DIR / "dataset_limpio_v2.parquet"

limpio_v2.to_csv(csv_path, index=False)
try:
    limpio_v2.to_parquet(parquet_path, index=False)
    print("Guardado CSV y Parquet:", csv_path.as_posix(), parquet_path.as_posix())
except Exception as e:
    print("Guardado CSV, Parquet no disponible:", csv_path.as_posix(), "|", e)

print("\nMuestra final:")
print(limpio_v2.head())


## Checklist final

- Tipos normalizados (`monto` numérico, `fecha` UTC, textos homogéneos).
- Nulos tratados: canales imputados, montos negativos a `NaN`.
- Catálogo aplicado a `categoria` → `categoria_final`.
- Enriquecimiento con `clientes` validado `m:1`.
- Duplicados removidos en `venta_id` y catálogo deduplicado.
- Datos guardados en `professor/pandas_v2/data/clean/` como `dataset_limpio_v2.*`.

Siguiente: usa el notebook 05 para EDA con `groupby`, `pivot_table` y `MultiIndex`.
